# Server

In [16]:
import tekore as tk
from dotenv import load_dotenv

load_dotenv()

conf = tk.config_from_environment()
cred = tk.Credentials(*conf)
spotify = tk.Spotify()
host = '127.0.0.1'
port = 5000

In [23]:
from os import environ as env
import json
from authentication.spotify_server import SpotifyServer
try:
    with open(".spotify_credentials.json", "r") as infile:
        token_dict = json.load(infile)
        token_dict["scope"] = " ".join(token_dict["scope"])
        spotify.token = tk.Token(token_dict, token_dict["uses_pkce"])
        spotify.token = cred.refresh(spotify.token)
except Exception as e:
    print(e)
    app = SpotifyServer(host, port, spotify, cred)
    spotify.token = app.spawn_single_use_server()

In [28]:
from typing import List
token: tk.Token = spotify.token
scopes: List[str] = list(token.scope)
token_dict = {
    "token_type": token.token_type,
    "access_token": token.access_token,
    "refresh_token": token.refresh_token,
    "scope": scopes[0].replace("[", "").replace("]", "").replace("'", "").split(", ") if len(scopes) == 0 else scopes,
    "expires_at": token.expires_at,
    "uses_pkce": token.uses_pkce,
    "expires_in": 0
}
json_object = json.dumps(token_dict, indent=4)
with open(".spotify_credentials.json", "w") as outfile:
    outfile.write(json_object)

# Spotify API

In [29]:
spotify.token = cred.refresh(spotify.token)

Scope('user-read-currently-playing', 'user-top-read')

In [7]:
from project.random_track import RandomTrack

rndTrack = RandomTrack(spotify, cred)
tracks = rndTrack.random_tracks()
for track in tracks:
    print(track.name)

Athen
I Ain't Worried
growing up (feat. remme & alicja)
All the Way (I Believe In Steve)
Am Ende
Another Heart
Alles wegen Dir
abcdefu
All Good
We Don't Talk About Bruno
Enemy (with JID) - from the series Arcane League of Legends
AVA
Fallin’ (Adrenaline)
Acapulco
Die Nacht von Freitag auf Montag
ave cesaria
Ain't No Rest For The Wicked - Original Version
Parle à ta tête
As It Was
Africa
Sweet Dreams (Are Made of This) - Remastered
Another Love
STAR WALKIN' (League of Legends Worlds Anthem)
Anti-Hero
Alright (feat. KIDDO)
Auf & Ab
Where Are You Now
All I Want for Christmas Is You
Alles neu
Atlantis
Have It All
Kernkraft 400 (A Better Day)
Wonderful Dream (Holidays Are Coming) - Radio Version
Bad Memories (feat. Elley Duhé & FAST BOY)
Apocalypse
Another Love
Bamba (feat. Aitch & BIA)
Anti-Hero
Santa Tell Me
Bamba (feat. Aitch & BIA)
Rockin' Around The Christmas Tree
As You Are
ALLEINE - SW43 L33 D1DO REMIX
All Girls Are The Same
Comment te dire adieu
It's Beginning to Look a Lot Like Chr

In [8]:
from project.util import Partition

artist_id_limit=50
artist_ids = [artist.id for track in tracks for artist in track.artists]
artist_id_partitions = Partition(artist_ids)
artists = [spotify.artists(ids) for ids in artist_id_partitions]
artists = [artist for partition in artists for artist in partition]
artists

[FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [1 x str]
   href = 'https://api.spotify.com/v1/artists/7jQYzUpPshdP6jZS7lJSDU'
   id = '7jQYzUpPshdP6jZS7lJSDU'
   images = [3 x Image(height, url, width)]
   name = 'Georg Stengel'
   popularity = 50
   type = 'artist'
   uri = 'spotify:artist:7jQYzUpPshdP6jZS7lJSDU',
 FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [2 x str]
   href = 'https://api.spotify.com/v1/artists/5Pwc4xIPtQLFEnJriah9YJ'
   id = '5Pwc4xIPtQLFEnJriah9YJ'
   images = [3 x Image(height, url, width)]
   name = 'OneRepublic'
   popularity = 85
   type = 'artist'
   uri = 'spotify:artist:5Pwc4xIPtQLFEnJriah9YJ',
 FullArtist with fields:
   external_urls = {'spotify'}
   followers = Followers(href, total)
   genres = [1 x str]
   href = 'https://api.spotify.com/v1/artists/0ql0Aw24KJaMeOVlwadzFE'
   id = '0ql0Aw24KJaMeOVlwadzFE'
   images = [3 x Image(height

In [9]:
import pylast
from authentication.lastfm_credentials import LastFmCredentials
from lastfm import LastFmScraper, LastFmProxy

creds = LastFmCredentials(env["LASTFM_API_KEY"], env["LASTFM_SHARED_SECRET"])
last_network = pylast.LastFMNetwork(
    api_key=creds.api_key,
    api_secret=creds.shared_secret,
)

lastfm = LastFmProxy(last_network, LastFmScraper())

In [10]:
import track as model
analyzed_tracks = model.AnalyzedTracks(tracks, artists)

In [11]:
from project.track import TaggedTrack
# TODO: fix empty artists
for track in analyzed_tracks.tracks:
    try:
        tags = TaggedTrack(lastfm, track.name, track.artist_names).tags()
        if len(tags) == 0:
            raise Exception("No tags")
    except Exception as e:
        print(e, track.id, track.name, track.artist_names)
        continue
    track.tags = tags

No tags 2X5gs8HvIDmZQ2rlWmkscx Athen ['Georg Stengel']
No tags 3Yww1k7Up2ja2ytfmXjJp8 Wonderful Dream (Holidays Are Coming) - Radio Version []
No tags 3rb0tMq42WfggucPm0HHkA Bad Memories (feat. Elley Duhé & FAST BOY) []
No tags 0yc6Gst2xkRu0eMLeRMGCX Apocalypse []
No tags 6Hh6pxNSScePI9zjKBIUg2 Another Love []
No tags 7nFJSfGvvV0WjatTZN6QYi Bamba (feat. Aitch & BIA) []
No tags 0lizgQ7Qw35od7CYaoMBZb Santa Tell Me []
No tags 7sJNeWTXkZCEC7OnFMpGWL Bamba (feat. Aitch & BIA) []
No tags 2EjXfH91m7f8HiJN1yQg97 Rockin' Around The Christmas Tree []
No tags 6qwedJNVRAS1gNyK4WQIlx As You Are []
No tags 7dq9QIXx5eIz6w2MNzTan4 ALLEINE - SW43 L33 D1DO REMIX []
No tags 4VXIryQMWpIdGgYR4TrjT1 All Girls Are The Same []
No tags 5tYSTEEJGs5Vno1OchhVis Comment te dire adieu []
No tags 2pXpURmn6zC5ZYDMms6fwa It's Beginning to Look a Lot Like Christmas (with Mitchell Ayres & His Orchestra) []
No tags 77khP2fIVhSW23NwxrRluh A Holly Jolly Christmas []
No tags 5a1iz510sv2W9Dt1MvFd5R It's Beginning to Look a 

# Firebase

In [12]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.ApplicationDefault()
firebase_admin.initialize_app()

In [13]:
from google.cloud.firestore_v1 import Client

firestore_client: Client = firestore.client()
analyzed_tracks.upsert(firestore_client.collection("tracks"))